In [155]:
from datetime import datetime

import numpy as np
import yfinance as yf

np.float_ = np.float64
from prophet import Prophet

import pandas as pd


# Função para coletar dividendos históricos
def coletar_dividendos(ticker):
    acao = yf.Ticker(ticker)
    dividendos = acao.dividends
    dividendos = dividendos.reset_index()
    dividendos.columns = ['ds', 'y']
    dividendos['ds'] = pd.to_datetime(dividendos['ds']).dt.tz_localize(None)
       
    return dividendos


# Função para coletar dados do índice e calcular a média anual
def calcular_taxa_retorno_bvsp():
    bvsp = yf.Ticker('^BVSP')
    historico = bvsp.history(period='10y')  # Dados dos últimos 10 anos
    historico['Year'] = historico.index.year
    anual = historico.groupby('Year')['Close'].mean()  # Média anual
    retorno_medio_anual = anual.pct_change().mean()  # Taxa de retorno anual média
    return retorno_medio_anual


# Função para calcular a taxa de crescimento dos dividendos usando Prophet
def calcular_taxa_crescimento_dividendos(ticker):
    dividendos = coletar_dividendos(ticker)

    # Criar e ajustar o modelo Prophet
    modelo = Prophet(growth='linear', yearly_seasonality=False, weekly_seasonality=False, daily_seasonality=False)
    modelo.fit(dividendos)

    # Prever para o futuro e obter a taxa de crescimento
    futuro = modelo.make_future_dataframe(periods=1, freq='YE')  # Adiciona um ano
    previsoes = modelo.predict(futuro)
    
    previsoes_por_ano = previsoes.groupby(pd.Grouper(key='ds', freq='YE')).sum()['trend']
    taxa_crescimento = previsoes_por_ano[previsoes_por_ano > 0].pct_change().mean()
    
    dividendos = previsoes.iloc[:, :2]
    dividendos.columns = ['ds', 'y']

    return dividendos, taxa_crescimento


def calcular_valor_justo(ticker):
    """
    Calcula o valor justo da ação usando o Modelo de Gordon.
    """
    taxa_retorno = calcular_taxa_retorno_bvsp() + 0.06
    dividendos, taxa_crescimento = calcular_taxa_crescimento_dividendos(ticker)

    print(f'Taxa de retorno média anual do ^BVSP: {taxa_retorno:.4f}')
    print(f'Taxa de crescimento dos dividendos para {ticker}: {taxa_crescimento:.4f}')

    # Definir a data atual
    data_atual = dividendos['ds'].max()
    
    # Filtrar os dados para os últimos 12 meses
    filtro = (dividendos['ds'] >= data_atual - pd.DateOffset(months=12))
    df_ultimos_12_meses = dividendos[filtro]
    
    # Calcular a soma dos valores
    dividendos_ultimos_12_meses = df_ultimos_12_meses['y'].sum()
    print(f'Dividendos dos últimos 12 meses de {ticker} é: {dividendos_ultimos_12_meses:.2f}')

    # Calcular o valor justo usando o Modelo de Gordon
    valor_justo = dividendos_ultimos_12_meses * (1 + taxa_crescimento) / abs(taxa_retorno - taxa_crescimento)

    return valor_justo

In [157]:
# Exemplo de uso
ticker = 'EGIE3.SA'  # Exemplo de ticker para dividendos
valor_justo = calcular_valor_justo(ticker)

print(f'O valor justo da ação {ticker} é: {valor_justo:.2f}')

00:44:00 - cmdstanpy - INFO - Chain [1] start processing
00:44:00 - cmdstanpy - INFO - Chain [1] done processing


Taxa de retorno média anual do ^BVSP: 0.1538
Taxa de crescimento dos dividendos para EGIE3.SA: 0.2137
Dividendos dos últimos 12 meses de EGIE3.SA é: 1.77
O valor justo da ação EGIE3.SA é: 35.84
